In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
%%bigquery df_ga_sample --project ibento-prod
SELECT date, visitId, if(totals.transactions > 0, 1, 0) transaction_flag
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170101' AND '20170301'

#https://console.cloud.google.com/bigquery?referrer=search&hl=es&project=ibento-prod&ws=!1m5!1m4!4m3!1sbigquery-public-data!2sgoogle_analytics_sample!3sga_sessions_20170801

Query is running:   0%|          |

Downloading:   0%|          |

In [4]:
df_ga_sample['date'] = pd.to_datetime(df_ga_sample['date'])

In [5]:
df_ga_sample.head()

,date,visitId,transaction_flag
0,2017-01-14,1484438078,0
1,2017-01-14,1484463010,0
2,2017-01-14,1484413323,0
3,2017-01-14,1484454359,0
4,2017-01-14,1484434947,0


In [6]:
df_daily = df_ga_sample.groupby('date').agg({'visitId': ['nunique'], 'transaction_flag': ['sum']})
df_daily.columns = ['sessions', 'transactions']
df_daily.reset_index(inplace=True)
df_daily.head()

,date,sessions,transactions
0,2017-01-01,1353,11
1,2017-01-02,1598,18
2,2017-01-03,2367,29
3,2017-01-04,2351,29
4,2017-01-05,2144,41


In [10]:


# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_daily.date, y=df_daily.sessions, name="sessions", marker_color="blue", opacity= .5),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_daily.date, y=df_daily.transactions, name="transactions"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Sessions & Transactions by date - Google Analytics Sample"
)

# Set x-axis title
fig.update_xaxes(title_text="date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> sessions", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> transactions", secondary_y=True)

fig.show()

In [12]:
df_daily['conversion'] = df_daily.apply(lambda x: round(x.transactions/x.sessions, 4), axis=1)

In [15]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=df_daily.date, y=df_daily.sessions, name="sessions", marker_color="blue", opacity= .4),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_daily.date, y=df_daily.conversion, name="conversion", marker_color="green"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Sessions & Conversion Rate by date - Google Analytics Sample"
)

# Set x-axis title
fig.update_xaxes(title_text="date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> sessions", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> conversion", secondary_y=True)

fig.show()

In [58]:
df_daily.describe()

,sessions,transactions,conversion
count,60.00000,60.0,60.000000
mean,2119.20000,23.95,0.010938
std,399.33117,11.216551,0.004065
min,1353.00000,4.0,0.002500
25%,1761.75000,14.75,0.007900
50%,2180.50000,25.5,0.010950
75%,2339.75000,29.0,0.012825
max,3524.00000,67.0,0.024700
